In [ ]:
#Advent of Code Problem 4
#...I dont know why I wasnt using functions before...
#of course we now have two data type in one input....

import requests
import numpy as np


def getPuzzleInputAsList(day): #define our function
    cookie_file = open("../keys/session.cookie") #open the keys file, this keeps us from having it in static code
    cookie_raw = cookie_file.readline() #read the keys file, leads to more secure sharing and fs controls
    cookie = cookie_raw.strip("\n") #strip the newline char
    sesh = requests.Session() #initialize our session
    url = "https://adventofcode.com/2021/day/"+str(day)+"/input" #append our day object to our destination url
    req = sesh.get(url,cookies={"session":cookie}) #make our request
    input_raw = req.text #get our request text
    input_list = input_raw.split("\n") #split our request text based on newline char
    return input_list

#parse winning numbers out into a flat array
def getWinningNums(input_list):
    raw_winning = input_list[0]
    winning = raw_winning.split(",")
    return np.array(winning,dtype=np.int64)

#parse boards into a 3d shape
def getGameBoards(input_list):
    raw_gameboards = np.delete(input_list,np.argwhere(input_list == '')) #litty: https://thispointer.com/delete-elements-from-a-numpy-array-by-value-or-conditions-in-python/
    gameboards = np.array([],dtype=np.int64)
    for ea in raw_gameboards:
        gameboard_spaces = np.array(ea.split(" "))
        gameboard_num = np.array(np.delete(gameboard_spaces,np.argwhere(gameboard_spaces == '')),dtype=np.int64)
        gameboards = np.append(gameboards,gameboard_num)
    
    gameboards = np.reshape(gameboards,(-1,5,5))
    return gameboards

#data was coming in as string so this wasnt working, parsed data to int during GB creation; works on entire array now
def replaceWinner(gameboard,winner):
    gameboard = np.where(gameboard == winner, 0, gameboard)
    return gameboard

#https://stackoverflow.com/questions/69601220/bingo-using-python
#tests for lines of 0's in bingo card
def isWinner(gameboard):
    for i in range (5):
        row_zeros=np.count_nonzero(gameboard[i:,])#would like to understand this : notation
        col_zeros=np.count_nonzero(gameboard[:,i])
        if not row_zeros or not col_zeros: #check if we have 0 non-zeros, if line with non-zeros is discovered continue
            return (True)
    diagonal_zeros=np.count_nonzero(np.diag(gameboard))
    diagonal1_zeros=np.count_nonzero(np.diag(np.fliplr(gameboard)))
    if not diagonal_zeros or not diagonal1_zeros:
        return(True)
    return(False)


def bingoSubsystem(gameboards,numbers):
    winner = False
    winning_board = np.array([])
    for ea in winning_nums:
        win_num = ea
        if winner:
            break
        board_counter = 0
        gameboards = replaceWinner(gameboards,win_num)
        for ea in gameboards:
            winner = isWinner(ea)
            if winner:
                winning_board_index = board_counter
                winning_board = ea
                return win_num,winning_board_index,winning_board
            board_counter+=1
    return "No winning board??"

input_list = getPuzzleInputAsList(4) # this is done to reduce stress on AoC servers, only one call is needed

gameboards = getGameBoards(np.delete(input_list,0))#gets game boards after removing winning numbers from input_list

winning_nums = getWinningNums(input_list)

winning_details = bingoSubsystem(gameboards,winning_nums)
raw_winning_board = gameboards[winning_details[1]]
marked_winning_board = winning_details[2]
winning_number = winning_details[0]
print(winning_number)
print(raw_winning_board)
print(marked_winning_board)
print(winning_number*np.sum(marked_winning_board))

In [ ]:
#Advent of Code Problem 4 part 2
#...I dont know why I wasnt using functions before...
#of course we now have two data type in one input....

#pretty sure I am running into an async issue, not all the winning game boards are being removed from the array
#this is leading to a false positive of a final gameboard when doing it the way I am
#we are also getting some game boards removed from the wrong indexes because of this..

#wont lie, looked up some solutions on reddit and damn.. 4d arrays that show every state... 
#I dont quite even know how they derive their solutions but their code is ~12 lines...
#I definately have a lot to learn about python, maybe all languages lol
#perhaps this is why I should have tried in c/java despite the compilation issue, easier async manipulation??
#jupyter is fun and its a great learning experience!!

#going to try to convert to np.nditer and see if that improves performance, if that doesnt work rewrite it with shear np


import requests
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)


def getPuzzleInputAsList(day): #define our function
    cookie_file = open("../keys/session.cookie") #open the keys file, this keeps us from having it in static code
    cookie_raw = cookie_file.readline() #read the keys file, leads to more secure sharing and fs controls
    cookie = cookie_raw.strip("\n") #strip the newline char
    sesh = requests.Session() #initialize our session
    url = "https://adventofcode.com/2021/day/"+str(day)+"/input" #append our day object to our destination url
    req = sesh.get(url,cookies={"session":cookie}) #make our request
    input_raw = req.text #get our request text
    input_list = input_raw.split("\n") #split our request text based on newline char
    return input_list

#parse winning numbers out into a flat array
def getWinningNums(input_list):
    raw_winning = input_list[0]
    winning = raw_winning.split(",")
    return np.array(winning,dtype=np.int64)

#parse boards into a 3d shape
def getGameBoards(input_list):
    raw_gameboards = np.delete(input_list,np.argwhere(input_list == '')) #litty: https://thispointer.com/delete-elements-from-a-numpy-array-by-value-or-conditions-in-python/
    gameboards = np.array([],dtype=np.int64)
    for ea in raw_gameboards:
        gameboard_spaces = np.array(ea.split(" "))
        gameboard_num = np.array(np.delete(gameboard_spaces,np.argwhere(gameboard_spaces == '')),dtype=np.int64)
        gameboards = np.append(gameboards,gameboard_num)
    
    gameboards = np.reshape(gameboards,(-1,5,5))
    return gameboards

#data was coming in as string so this wasnt working, parsed data to int during GB creation; works on entire array now
def replaceWinner(gameboard,winner):
    gameboard = np.where(gameboard == winner, 0, gameboard)
    return gameboard

#https://stackoverflow.com/questions/69601220/bingo-using-python
#tests for lines of 0's in bingo card, well great.. zero is one of our numbers..
def isWinner(gameboard):
    for i in range (5):
        row_zeros=np.count_nonzero(gameboard[i:,])#would like to understand this : notation
        col_zeros=np.count_nonzero(gameboard[:,i])
        if not row_zeros or not col_zeros: #check if we have 0 non-zeros, if line with non-zeros is discovered continue
            return (True)
    diagonal_zeros=np.count_nonzero(np.diag(gameboard))
    diagonal1_zeros=np.count_nonzero(np.diag(np.fliplr(gameboard)))
    if not diagonal_zeros or not diagonal1_zeros:
        return(True)
    return(False)


async def removeGameboards(gameboards):
    winning_board_indices = np.array([],dtype=np.int64)
    board_counter = 0
    for ea in gameboards:
        winner = isWinner(ea)
        if winner: #recall bingosubsystem with removal of winner?? led to async issues when checking boards.. So tried this
            winning_board_indices = np.append(winning_board_indices,board_counter)
            #print("winner at: "+str(board_counter)+", adding to indices: ")
            #print(winning_board_indices)
            #print(ea)
            #print(gameboards[board_counter])
            #gameboards = np.delete(gameboards,board_counter,axis=0)
            #print(gameboards)
        board_counter+=1
    return np.delete(gameboards,winning_board_indices,axis=0)

async def bingoSubsystem(gameboards,numbers):
    winner = False
    winning_board = np.array([],dtype=np.int64)
    winning_board_indices = np.array([],dtype=np.int64)
    win_num = -1
    for win_num in winning_nums:
        print(np.shape(gameboards),win_num)
        if np.shape(gameboards)[0]<=1:
            print("returning")
            print(gameboards)
            return win_num,winning_board_indices,winning_board
        gameboards = replaceWinner(gameboards,win_num)
        print("clearing gameboards")
        print(gameboards)
        gameboards = await removeGameboards(gameboards)
        print(gameboards)
        
    print("finished our numbers")
    return win_num,winning_board_indices,winning_board

input_list = getPuzzleInputAsList(4) # this is done to reduce stress on AoC servers, only one call is needed

gameboards = getGameBoards(np.delete(input_list,0))#gets game boards after removing winning numbers from input_list

winning_nums = getWinningNums(input_list)

print(await bingoSubsystem(gameboards,winning_nums))
#print(winning_details)


In [ ]:
#used as a notepad again lol

def bingoSubsystem(gameboards,numbers):
    winner = False
    winning_board = np.array([],dtype=np.int64)
    winning_board_indices = np.array([],dtype=np.int64)
    win_num = -1
    for ea in winning_nums:
        win_num = ea
        print(np.shape(gameboards))
        print(winner)
        if np.shape(gameboards)[0] == 1:
            return win_num,winning_board_index,winning_board
        if winner:
            print("removing board at index:"+str(winning_board_indices))
            #print(gameboards)
            gameboards = np.delete(gameboards,0,axis=0)
            print(gameboards)
            winning_board_indices = np.array([],dtype=np.int64)
            winner = False
        board_counter = 0
        gameboards = replaceWinner(gameboards,win_num)
        for ea in gameboards:
            winner = isWinner(ea)
            if winner:
                print("wining ticket at: "+str(board_counter))
                winning_board_indices = np.append(winning_board_indices,board_counter)
                print(winning_board_indices)
            board_counter+=1
    print("finished our numbers")
    return win_num,winning_board_indices,winning_board



for ea in gameboards:
            print("testing for winner: "+str(board_counter))
            print(ea)
            winner = isWinner(ea)
            print(winner)
            if winner: #recall bingosubsystem with removal of winner??
                print(gameboards)
                print("winner at: "+str(board_counter)+", deleting board: ")
                print(ea)
                print(gameboards)
                #return bingoSubsystem(np.delete(gameboards,board_counter,axis=0),winning_nums)
            board_counter+=1